In [1]:
import datasets

In [2]:
xglue = datasets.load_dataset('xglue', 'ner')

Reusing dataset x_glue (/Users/spenno_fr/.cache/huggingface/datasets/x_glue/ner/1.0.0/9d181dd4413e6dfb0502b603c5ced10d2988d7bc97dd4c899eabee4397234139)


In [3]:
xglue.shape

{'train': (14042, 2),
 'validation.en': (3252, 2),
 'validation.de': (2874, 2),
 'validation.es': (1923, 2),
 'validation.nl': (2895, 2),
 'test.en': (3454, 2),
 'test.de': (3007, 2),
 'test.es': (1523, 2),
 'test.nl': (5202, 2)}

In [4]:
xglue['train'][-10]

{'words': ['Peter',
  'Hedblom',
  '(',
  'Sweden',
  ')',
  '70',
  '75',
  '75',
  ',',
  'Retief',
  'Goosen',
  '(',
  'South'],
 'ner': [2, 2, 0, 6, 0, 0, 0, 0, 0, 2, 2, 0, 6]}

In [8]:
test_sentence = xglue['train']['words'][0]

In [6]:
from transformers import BertTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
# help(tokenizer)

In [12]:
# Tokenize the sample test sentence intp a numpy array with HuggingFace BertTokenizer
tokenizer(test_sentence, padding="longest", truncation=True, max_length=512, return_tensors="np")

{'input_ids': array([[  101,  7327,   102],
       [  101, 19164,   102],
       [  101,  2446,   102],
       [  101,  2655,   102],
       [  101,  2000,   102],
       [  101, 17757,   102],
       [  101,  2329,   102],
       [  101, 12559,   102],
       [  101,  1012,   102]]), 'token_type_ids': array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]]), 'attention_mask': array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])}